<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/TwelveLabs_Quickstart_Search.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Search
This guide shows how to utilize the TwelveLabs Python SDK for searching within your video content.

# Prerequisites
Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project.
- The videos you wish to upload must meet the requirements in the [Prerequisites](https://docs.twelvelabs.io/docs/upload-single-videos#prerequisites) section of the **Upload single videos page**.



# Procedure

## Install the TwelveLabs Python SDK

In [4]:
%pip install -U -q twelvelabs

## Import the required packages

In [5]:
import os
from glob import glob
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

## Configure your API key


In [8]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get('TL_API_KEY')

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Create an index

An index is a basic unit for organizing and storing video data consisting of video embeddings and metadata. Indexes facilitate information retrieval and processing.

When creating a new index, you must specify at least the following information:
- **Name**: Use a brief and descriptive name to facilitate future reference and management.
- **Model configuration**: Provide a list containing the video understanding models and the associated model options you want to enable.

TwelveLabs provides two distinct model types:

- **Embedding models (Marengo)**: This model performs tasks such as search and embedding generation , enabling enhanced video understanding.
- **Generative models (Pegasus)**: This model generates text based on your videos.

Model options determine what information the video understanding model will process:
- **visual**: Analyzes visual content, including actions, objects, events, text (OCR), and brand logos.
- **audio**: Analyzes audio content, including ambient sounds and human speech.

In [9]:
client = TwelveLabs(api_key=TL_API_KEY)

index = client.index.create(
    name="Temp-1",
    models=[
        {
            "name": "marengo2.7",
            "options": ["visual", "audio"],
        }
    ]
)
print(f"Created index: id={index.id} name={index.name} models={index.models}")


Created index: id=68261cea7e5c903c4f66fc29 name=Temp-1 models=root=[Model(name='marengo2.7', options=['visual', 'audio'], addons=None, finetuned=False)]


## Upload videos to your index

In [10]:
# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "video_12_labs.mp4"  # In a colab Notebook, just upload the file in the root folder and just give the filename.

video_files = glob(VIDEO_PATH)
if(len(video_files)==0):
  raise Exception("Video file was not provided")
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file)
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")


Uploading video_12_labs.mp4
Created task: id=68261d52cd88e73ccc6241e6
  Status=indexing
  Status=ready
Uploaded video_12_labs.mp4. The unique identifer of your video is 68261d52cd88e73ccc6241e6.


## Perform searches

**Using text queries**:

In [11]:
# Search Query
search_results = client.search.query(index_id=index.id, query_text="who is the Graphic Designer", options=["visual", "audio"])

# Utility function to print a specific page
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )
print_page(search_results.data)
while True:
    try:
        print_page(next(search_results))
    except StopIteration:
        break

 video_id=68261d52cd88e73ccc6241e6 score=84.49 start=0.0 end=8.438888804 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.26 start=67.066665996 end=83.466665832 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.18 start=8.438888804 end=16.877777608 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.93 start=16.877777608 end=42.19444402 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.93 start=230.255553253 end=278.655552769 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.92 start=58.866666078 end=67.066665996 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.91 start=101.99999898 end=107.166665595 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.9 start=287.633330457 end=294.576665278 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.77 start=83.499999165 end=101.966665647 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.65 start=215.833331175 end=230.255553253 confidence=high
 video_id=68261

**Using image queries**

In [12]:
search_results = client.search.query(
    index_id=index.id,
    query_media_type="image",
    query_media_file="Test_image.png", # Use query_media_url instead to provide a file from a publicly accessible URL.
    options=["visual"]
)

# Utility function to print a specific page
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )

print_page(search_results.data)

while True:
    try:
        print_page(next(search_results))
    except StopIteration:
        break

 video_id=68261d52cd88e73ccc6241e6 score=86.55 start=151.066665156 end=170.499998295 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=85.27 start=137.499998625 end=151.033331823 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=85.27 start=215.833331175 end=237.466664292 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.75 start=170.533331628 end=215.799997842 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.42 start=107.199998928 end=137.499998625 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.35 start=58.866666078 end=101.966665647 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.32 start=237.499997625 end=260.733330726 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=84.05 start=8.438888804 end=58.866666078 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.28 start=0.0 end=8.438888804 confidence=high
 video_id=68261d52cd88e73ccc6241e6 score=83.27 start=260.766664059 end=287.599997124 confidence=high
 video